# Analyzing Aggravated Burglaries in Davidson County

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from IPython.display import Image
import json
import geopandas as gpd
from io import StringIO

Create a choropleth showing the number of burglaries per 1000 residents for each census tract.

Finally, we'll build some statistical models to see how well we can explain the number of aggravated burglaries using the median income of each census tract. For this, we'll be using the Generalized Linear Models module of the statsmodels library.

a. Build a "base model" - a Poisson regression model with just an intercept term with target variable the rate of burglaries per census tract. (Offset using the [log of the] population so that we are looking at the rate of burglaries per population instead of the number of burglaries.)

b. Now, build a Poisson regression model with target variable the rate of burglaries and predictor variable the median income. (Don't forget to offset by the population).

c. Finally, try out a negative binomial model. To get started with a negative binomial model, you can check out this tutorial.

d. How do your models compare? Hint: the fit models have an AIC attribute.

Prepare a short presentation (<10 minutes) of your findings.

Additional Resources for Generalized Linear Models:

DataCamp - Generalized Linear Models in Python
Beyond Multiple Linear Regression, Chapter 4 Warning - the code in this book is all R, but the conceptual explanations are very clear.
This set of notes, which talks about the problem of overdispersion.

1. A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. Make use of the API to find all aggravated burglary incidents that were reported during the six month period from January 1, 2021 through June 30, 2021.

In [2]:
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.geojson'
params = {'offense_description':'BURGLARY- AGGRAVATED',
              '$where':'incident_occurred between \'2021-01-01\' and \'2021-06-30\'',
              '$limit':2000
              }
response = requests.get(endpoint, params = params)

In [3]:
agg_burg = gpd.read_file(StringIO(response.text))

In [4]:
# drop duplicate rows with the same incident_number
agg_burg=agg_burg.drop_duplicates(subset = ['incident_number'])

2. Download the 2019 census tract shapefiles for Tennessee from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html. (The FIPS code for Tennessee is 47). Perform a spatial join to determine the census tract in which each burglary incident occurred. Which census tract had the highest number of burglaries? Warning - each incident can appear multiple times if there are multiple victims, so be sure that you aren't double-counting any incidents.

In [5]:
census_tract=gpd.read_file('../data/tl_2019_47_tract/')

In [99]:
census_tract['NAMELSAD']

0       Census Tract 210.02
1       Census Tract 209.01
2          Census Tract 203
3       Census Tract 209.02
4          Census Tract 208
               ...         
1492    Census Tract 221.12
1493    Census Tract 505.01
1494    Census Tract 506.01
1495    Census Tract 506.02
1496    Census Tract 505.02
Name: NAMELSAD, Length: 1497, dtype: object

In [7]:
burg_census = gpd.sjoin(agg_burg, census_tract, op = 'within')

/var/folders/h3/4dwpklsd0vq23n7lrrx_lryr0000gn/T/ipykernel_17235/1384890642.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  burg_census = gpd.sjoin(agg_burg, census_tract, op = 'within')


In [8]:
burg_census['NAMELSAD'].value_counts().index[0]

'Census Tract 163'

3. For this part, you'll need to request a census API key. Using the 2019 American Community Survey API, obtain, for each census tract, the population (B01001_001E in the detailed tables) and the median income (S1901_C01_012E in the subject tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37. Merge this new data with the burglaries data above.

In [9]:
with open('../data/census.json') as fi:
    credentials = json.load(fi)
api_key = credentials['api_key']

In [78]:
# population in the detailed tables
endpoint = 'https://api.census.gov/data/2019/acs/acs5'
params = {
          'get': 'NAME,B01001_001E',
          'for':'tract:*',
          'in' : ['state:47', 'county:037'],
          'key':api_key
        }
response = requests.get(endpoint, params = params)

#get=NAME,B01001_001E&for=tract:*&in=state:01&in=county:*&key=YOUR_KEY_GOES_HERE

In [79]:
print(response.url)

https://api.census.gov/data/2019/acs/acs5?get=NAME%2CB01001_001E&for=tract%3A%2A&in=state%3A47&in=county%3A037&key=a2b3ec937c5c5e437b9e4395287b5d91d1539fc3


In [80]:
endpoint

'https://api.census.gov/data/2019/acs/acs5'

In [81]:
params

{'get': 'NAME,B01001_001E',
 'for': 'tract:*',
 'in': ['state:47', 'county:037'],
 'key': 'a2b3ec937c5c5e437b9e4395287b5d91d1539fc3'}

In [82]:
response.text

'[["NAME","B01001_001E","state","county","tract"],\n["Census Tract 156.31, Davidson County, Tennessee","12176","47","037","015631"],\n["Census Tract 158.04, Davidson County, Tennessee","4098","47","037","015804"],\n["Census Tract 177.01, Davidson County, Tennessee","2466","47","037","017701"],\n["Census Tract 177.02, Davidson County, Tennessee","5210","47","037","017702"],\n["Census Tract 183.01, Davidson County, Tennessee","8254","47","037","018301"],\n["Census Tract 183.02, Davidson County, Tennessee","2774","47","037","018302"],\n["Census Tract 184.09, Davidson County, Tennessee","4205","47","037","018409"],\n["Census Tract 184.10, Davidson County, Tennessee","3467","47","037","018410"],\n["Census Tract 110.02, Davidson County, Tennessee","2553","47","037","011002"],\n["Census Tract 143, Davidson County, Tennessee","1860","47","037","014300"],\n["Census Tract 154.02, Davidson County, Tennessee","4717","47","037","015402"],\n["Census Tract 154.05, Davidson County, Tennessee","4882","

In [66]:
response.json()

[['NAME', 'B01001_001E', 'state', 'county', 'tract'],
 ['Census Tract 156.31, Davidson County, Tennessee',
  '12176',
  '47',
  '037',
  '015631'],
 ['Census Tract 158.04, Davidson County, Tennessee',
  '4098',
  '47',
  '037',
  '015804'],
 ['Census Tract 177.01, Davidson County, Tennessee',
  '2466',
  '47',
  '037',
  '017701'],
 ['Census Tract 177.02, Davidson County, Tennessee',
  '5210',
  '47',
  '037',
  '017702'],
 ['Census Tract 183.01, Davidson County, Tennessee',
  '8254',
  '47',
  '037',
  '018301'],
 ['Census Tract 183.02, Davidson County, Tennessee',
  '2774',
  '47',
  '037',
  '018302'],
 ['Census Tract 184.09, Davidson County, Tennessee',
  '4205',
  '47',
  '037',
  '018409'],
 ['Census Tract 184.10, Davidson County, Tennessee',
  '3467',
  '47',
  '037',
  '018410'],
 ['Census Tract 110.02, Davidson County, Tennessee',
  '2553',
  '47',
  '037',
  '011002'],
 ['Census Tract 143, Davidson County, Tennessee',
  '1860',
  '47',
  '037',
  '014300'],
 ['Census Tract 15

In [67]:
pop=pd.read_json(response.text)

In [68]:
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       162 non-null    object
 1   1       162 non-null    object
 2   2       162 non-null    object
 3   3       162 non-null    object
 4   4       162 non-null    object
dtypes: object(5)
memory usage: 6.5+ KB


In [71]:
pop = pop[1:].rename(columns = {0:"name", 1:"population",2:'state',3:'county', 4:'track'})

In [75]:
pop

,name,population,state,county,track
2,"Census Tract 158.04, Davidson County, Tennessee",4098,47,037,015804
3,"Census Tract 177.01, Davidson County, Tennessee",2466,47,037,017701
4,"Census Tract 177.02, Davidson County, Tennessee",5210,47,037,017702
5,"Census Tract 183.01, Davidson County, Tennessee",8254,47,037,018301
6,"Census Tract 183.02, Davidson County, Tennessee",2774,47,037,018302
...,...,...,...,...,...
157,"Census Tract 156.13, Davidson County, Tennessee",5252,47,037,015613
158,"Census Tract 156.20, Davidson County, Tennessee",7262,47,037,015620
159,"Census Tract 161, Davidson County, Tennessee",2345,47,037,016100
160,"Census Tract 114, Davidson County, Tennessee",4617,47,037,011400


In [89]:
# median income
endpoint = 'https://api.census.gov/data/2019/acs/acs5/subject'
params = {
          'get': 'NAME,S1901_C01_012E',
          'for':'tract:*',
          'in' : ['state:47', 'county:037'],
          'key':api_key
        }
response = requests.get(endpoint, params = params)

In [92]:
response.json()

[['NAME', 'S1901_C01_012E', 'state', 'county', 'tract'],
 ['Census Tract 156.31, Davidson County, Tennessee',
  '75579',
  '47',
  '037',
  '015631'],
 ['Census Tract 158.04, Davidson County, Tennessee',
  '45129',
  '47',
  '037',
  '015804'],
 ['Census Tract 177.01, Davidson County, Tennessee',
  '134786',
  '47',
  '037',
  '017701'],
 ['Census Tract 177.02, Davidson County, Tennessee',
  '107813',
  '47',
  '037',
  '017702'],
 ['Census Tract 183.01, Davidson County, Tennessee',
  '87591',
  '47',
  '037',
  '018301'],
 ['Census Tract 183.02, Davidson County, Tennessee',
  '96576',
  '47',
  '037',
  '018302'],
 ['Census Tract 184.09, Davidson County, Tennessee',
  '65887',
  '47',
  '037',
  '018409'],
 ['Census Tract 184.10, Davidson County, Tennessee',
  '58028',
  '47',
  '037',
  '018410'],
 ['Census Tract 110.02, Davidson County, Tennessee',
  '49926',
  '47',
  '037',
  '011002'],
 ['Census Tract 143, Davidson County, Tennessee',
  '35313',
  '47',
  '037',
  '014300'],
 ['C

In [93]:
median_income = pd.read_json(response.text)

In [95]:
# delete the first row and rename the columns
median_income=median_income[1:].rename(columns = {0:"name", 1:"median_income",2:'state',3:'county', 4:'track'})

In [107]:
# merge pop with median_income
pop_income = pd.merge(pop,median_income, on = 'name')

In [112]:
# extract the Census Track from the name column of pop_income dataframe and save it to a new column NAMELSAD
pop_income['NAMELSAD']=pop_income['name'].apply(lambda x :x.split(',')[0])

In [113]:
pop_income

,name,population,state_x,county_x,track_x,median_income,state_y,county_y,track_y,NAMELSAD
0,"Census Tract 158.04, Davidson County, Tennessee",4098,47,037,015804,45129,47,037,015804,Census Tract 158.04
1,"Census Tract 177.01, Davidson County, Tennessee",2466,47,037,017701,134786,47,037,017701,Census Tract 177.01
2,"Census Tract 177.02, Davidson County, Tennessee",5210,47,037,017702,107813,47,037,017702,Census Tract 177.02
3,"Census Tract 183.01, Davidson County, Tennessee",8254,47,037,018301,87591,47,037,018301,Census Tract 183.01
4,"Census Tract 183.02, Davidson County, Tennessee",2774,47,037,018302,96576,47,037,018302,Census Tract 183.02
...,...,...,...,...,...,...,...,...,...,...
155,"Census Tract 156.13, Davidson County, Tennessee",5252,47,037,015613,37656,47,037,015613,Census Tract 156.13
156,"Census Tract 156.20, Davidson County, Tennessee",7262,47,037,015620,54149,47,037,015620,Census Tract 156.20
157,"Census Tract 161, Davidson County, Tennessee",2345,47,037,016100,48221,47,037,016100,Census Tract 161
158,"Census Tract 114, Davidson County, Tennessee",4617,47,037,011400,50857,47,037,011400,Census Tract 114


In [118]:
# merge agg_burg with pop_income on NAMELSAD column
burg_pop_income = pd.merge(burg_census,pop_income, on = 'NAMELSAD')